In [3]:
!pip install opencv-python

     |████████████████████████████████| 52.4MB 8.2MB/s eta 0:00:01


In [1]:
import pandas as pd 
import os
import numpy as np 
import cv2
from keras.preprocessing import image 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from skimage.segmentation import mark_boundaries 

Using TensorFlow backend.
Illegal line #2
	"(44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),
"
	in file "/Users/nobu/.matplotlib/stylelib/nobu-style.mplstyle"
Illegal line #3
	"(148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),
"
	in file "/Users/nobu/.matplotlib/stylelib/nobu-style.mplstyle"
Illegal line #4
	"(227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),
"
	in file "/Users/nobu/.matplotlib/stylelib/nobu-style.mplstyle"
Illegal line #5
	"(188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)])
"
	in file "/Users/nobu/.matplotlib/stylelib/nobu-style.mplstyle"
Bad val "cycler('color',[(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120)," on line #1
	"axes.prop_cycle: cycler('color',[(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),
"
	in file "/Users/nobu/.matplotlib/stylelib/nobu-style.mplstyle"
	Key axes.prop_cycle: 'cycler('color',[(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120)

In [2]:
def get_image_value(path, dim): 
    '''This function will read an image and convert to a specified version 
    and resize depending on which algorithm is being used. '''
    img = image.load_img(path, target_size = dim)
    img = image.img_to_array(img)
    return img/255

In [3]:
def get_img_array(img_paths, dim): 
    '''This fucntion takes a list of image paths and returns the np array 
    corresponding to each image.  It also takes the dim and whether edge is 
    specified in order to pass it to another function to apply these parameters.  
    This function uses get_image_value to perform these operations'''
    final_array = []
    from tqdm import tqdm
    for path in tqdm(img_paths):
        img = get_image_value(path, dim)
        final_array.append(img)
    final_array = np.array(final_array)  
    return final_array

In [4]:
def get_tts():
    '''This function will create a train test split'''  
    DIM =  (150,150) 
    np.random.seed(10)        
    gun_paths = [f'./Separated/FinalImages/Gun/{i}' for i in os.listdir('./Separated/FinalImages/Gun')] 
    gun_labels = [1 for i in range(len(gun_paths))]
    #human_paths = [f'./Separated/FinalImages/Human/{i}' for i in os.listdir('./Separated/FinalImages/Human')] 
    #human_labels = [2 for i in range(len(human_paths))]    
    others_paths = [f'./Separated/FinalImages/Others/{i}' for i in os.listdir('./Separated/FinalImages/Others')]
    np.random.shuffle(others_paths)
    others_paths = others_paths[:len(gun_paths)- 500]
    others_labels = [0 for i in range(len(others_paths))]

    np.random.shuffle(gun_paths)
    gun_paths = gun_paths[:len(gun_paths)+150]
    others_paths = others_paths[:len(others_paths)+150]

    gun_labels = [1 for i in range(len(gun_paths))]
    #human_labels = [2 for i in range(len(human_labels))]
    others_labels = [0 for i in range(len(others_paths))]
    #paths = gun_paths + human_paths + others_paths
    paths = gun_paths + others_paths
    labels = gun_labels + others_labels
    #labels = gun_labels + human_labels + others_labels
    x_train, x_test, y_train, y_test = train_test_split(paths, labels, stratify = labels, train_size = .90, random_state = 10)

    new_x_train = get_img_array(x_train, DIM)
    new_x_test = get_img_array(x_test, DIM)
    
    print('Train Value Counts')
    print(pd.Series(y_train).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('Test Value Counts')
    print(pd.Series(y_test).value_counts())
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Train Shape')
    print(new_x_train.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print('X Test Shape')
    print(new_x_test.shape)
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    y_train = np.array(y_train)
    y_test = np.array(y_test)
    y_test = to_categorical(y_test)
    y_train = to_categorical(y_train)
    tts = (new_x_train, new_x_test, y_train, y_test)
    return tts

x_train, x_test, y_train, y_test = get_tts()


100%|██████████| 325/325 [00:01<00:00, 277.06it/s]


Train Value Counts
1    2218
0     701
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Test Value Counts
1    247
0     78
dtype: int64
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Train Shape
(2919, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
X Test Shape
(325, 150, 150, 3)
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [5]:
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D, Dense, Dropout, Flatten 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd 
import matplotlib.pyplot as plt
import os
import pickle
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import cv2

In [6]:
def get_conv_model(dim = (150,150, 3)):
    '''This function will create and compile a CNN given the input dimension'''
    inp_shape = dim
    act = 'relu'
    drop = .25
    kernal_reg = regularizers.l1(.001)
    optimizer = Adam(lr = .0001)    
    model = Sequential() 
    model.add(Conv2D(64, kernel_size=(3,3),activation=act, input_shape = inp_shape, 
                     kernel_regularizer = kernal_reg,
                     kernel_initializer = 'he_uniform',  padding = 'same', name = 'Input_Layer'))
    model.add(MaxPooling2D(pool_size=(2, 2),  strides = (3,3)))
    model.add(Conv2D(64, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3))) 
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(Conv2D(128, (3, 3), activation=act, kernel_regularizer = kernal_reg, 
                     kernel_initializer = 'he_uniform',padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (3,3)))  
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(2, activation='sigmoid', name = 'Output_Layer'))
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    #model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model 

In [7]:
early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=10, min_delta = .00075)
model_checkpoint = ModelCheckpoint('ModelWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 2, mode = 'min')
epochs = 50
batch_size = 32
model = get_conv_model()
model_history = model.fit(x_train, y_train, batch_size = batch_size,
            epochs = epochs, 
     callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


Epoch 1/50
92/92 [==============================] - 43s 457ms/step - loss: 11.5079 - accuracy: 0.7850 - val_loss: 10.0921 - val_accuracy: 0.9077

Epoch 00001: val_loss improved from inf to 10.09211, saving model to ModelWeights.h5
Epoch 2/50
92/92 [==============================] - 42s 457ms/step - loss: 9.7774 - accuracy: 0.8979 - val_loss: 8.6829 - val_accuracy: 0.9600

Epoch 00002: val_loss improved from 10.09211 to 8.68294, saving model to ModelWeights.h5
Epoch 3/50
92/92 [==============================] - 44s 475ms/step - loss: 8.4324 - accuracy: 0.9330 - val_loss: 7.5323 - val_accuracy: 0.9508

Epoch 00003: val_loss improved from 8.68294 to 7.53226, saving model to ModelWeights.h5
Epoch 4/50
92/92 [==============================] - 44s 473ms/step - loss: 7.2705 - accuracy: 0.9581 - val_loss: 6.5049 - val_accuracy: 0.9692

Epoch 00004: val_loss improved from 7.53226 to 6.50490, saving model to ModelWeights.h5
Epoch 5/50
92/92 [==============================] - 43s 473ms/step - los

In [11]:
from tensorflow.keras.models import save_model
save_model(model, 'model_save')

/Users/nobu/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
/Users/nobu/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/base_layer.py:1397: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`layer.updates` will be removed in a future version. '



FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: model_save/assets


In [21]:
from tensorflow.keras.models import load_model
model = load_model('ModelWeights.h5')

In [22]:
!mkdir -p saved_model1

In [26]:
model.save_model('saved_model/my_model')

AttributeError: 'Sequential' object has no attribute 'save_model'

In [ ]:
def non_max_suppression(boxes, overlapThresh= .5):
    '''This image was taken from PyImageSearch... again cannot thank that guy enough'''
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
    # if the bounding boxes integers, convert them to floats --
    # this is important since we'll be doing a bunch of divisions
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
    # initialize the list of picked indexes	
    pick = []
    # grab the coordinates of the bounding boxes
    x1, y1, x2, y2 = boxes[:,0], boxes[:,1], boxes[:,2], boxes[:,3]    
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the bottom-right y-coordinate of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)
    # keep looping while some indexes still remain in the indexes
    # list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1, yy1, xx2, yy2 = np.maximum(x1[i], x1[idxs[:last]]), np.maximum(y1[i], y1[idxs[:last]]), np.minimum(x2[i], x2[idxs[:last]]), np.minimum(y2[i], y2[idxs[:last]])
        # compute the width and height of the bounding box
        w, h = np.maximum(0, xx2 - xx1 + 1), np.maximum(0, yy2 - yy1 + 1)
        # compute the ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        # delete all indexes from the index list that have
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
    # return only the bounding boxes that were picked using the
    # integer data type
    return pick

def get_img_prediction_bounding_box(path, model, dim):
    '''This function will create a bounding box over what it believes 
    is a weapon given the image path, dimensions, and model used to 
    detect the weapon.  Dimensions can be found within the Var.py file.  
    This function is still being used as I need to apply non-max suppresion 
    to create only one bounding box'''
    img = get_image_value(path, dim)   
    img = img.reshape(1, img.shape[0], img.shape[1], 3)
    pred = model.predict(img)[0]
    category_dict = {0: 'No Weapon', 1: 'Gun', 2: 'Human'}
    cat_index = np.argmax(pred)
    cat = category_dict[cat_index]
    print(f'{path}\t\tPrediction: {cat}\t{int(pred.max()*100)}% Confident')

    #speed up cv2
    cv2.setUseOptimized(True)
    cv2.setNumThreads(10) #change depending on your computer
    img = cv2.imread(path)
    clone = img.copy() 
    clone2 = img.copy()
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    ss.setBaseImage(img)
    ss.switchToSelectiveSearchFast()

    rects = ss.process() 
    windows = []
    locations = []
    print(f'Creating Bounding Boxes for {path}')
    for x, y, w,h in rects[:1001]: 
        startx, starty, endx, endy = x, y, x+w, y+h 
        roi = img[starty:endy, startx:endx]
        roi = cv2.resize(roi, dsize =dim, interpolation = cv2.INTER_CUBIC)
        windows.append(roi)
        locations.append((startx, starty, endx, endy))
    windows = np.array(windows)
    windows = windows.reshape(windows.shape[0], windows.shape[1], windows.shape[2], 3)
    windows = np.array(windows)
    locations = np.array(locations)
    predictions = model.predict(windows)
    nms = non_max_suppression(locations)
    bounding_cnt = 0
    for idx in nms:
        if np.argmax(predictions[idx]) != cat_index: 
            continue
        startx, starty, endx, endy = locations[idx]
        cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
        text = f'{category_dict[np.argmax(predictions[idx])]}: {int(predictions[idx].max()*100)}%'
        cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
        bounding_cnt += 1

    if bounding_cnt == 0: 
        pred_idx= [idx for idx, i in enumerate(predictions) if np.argmax(i) == cat_index]
        cat_locations = np.array([locations[i] for i in pred_idx])
        nms = non_max_suppression(cat_locations)
        if len(nms)==0:
            cat_predictions = predictions[:,cat_index]
            pred_max_idx = np.argmax(cat_predictions)
            pred_max = cat_predictions[pred_max_idx]
            pred_max_window = locations[pred_max_idx]
            startx, starty, endx, endy = pred_max_window
            cv2.rectangle(clone, (startx, starty), (endx, endy),  (0,0,255),2)
            text = f'{category_dict[cat_index]}: {int(pred_max*100)}%'
            cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)
        for idx in nms: 
            startx, starty, endx, endy = cat_locations[idx]
            cv2.rectangle(clone, (startx, starty), (endx, endy), (0,0,255), 2)
            text = f'{category_dict[np.argmax(predictions[pred_idx[idx]])]}: {int(predictions[pred_idx[idx]].max()*100)}%'
            cv2.putText(clone, text, (startx, starty+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,255,0),2)        
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    cv2.imshow(f'Test', np.hstack([clone, clone2]))
    cv2.waitKey(0)
    ss.clear()
    return predictions

#NORMAL MODEL
dim = (150, 150, 3)    
normal_model = get_conv_model(dim)
normal_model.load_weights('ModelWeights.h5') #path to the model weights
test_folder = 'neg' #folder where you will put your images to test
predictions = []
for idx, i in enumerate([i for i in os.listdir(test_folder) if i != 'ipynb_checkpoints']):
    img_path = f'{test_folder}/{i}'
    pred = get_img_prediction_bounding_box(img_path, normal_model, dim = (150,150))
    predictions.append(pred)

neg/189.jpg		Prediction: No Weapon	99% Confident
Creating Bounding Boxes for neg/189.jpg
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


In [ ]:
normal_model = get_conv_model(dim)
normal_model.load_weights('ModelWeights.h5') #path to the model weights

In [2]:
import os
#os.makedirs('./model', exist_ok=True)
from tensorflow.keras.models import load_model
model = load_model('ModelWeights.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)

[<KerasTensor: shape=(None, 2) dtype=float32 (created by layer 'Output_Layer')>]
[<KerasTensor: shape=(None, 150, 150, 3) dtype=float32 (created by layer 'Input_Layer_input')>]


In [18]:
import tensorflow as tf

pre_model = tf.keras.models.load_model("ModelWeights.h5")
pre_model.save("saved_model")
#tf.keras.models.save_model("ModelWeights.h5", 'saved_model1')


FOR DEVS: If you are overwriting _tracking_metadata in your class, this property has been used to save metadata in the SavedModel. The metadta field will be deprecated soon, so please move the metadata to a different file.
INFO:tensorflow:Assets written to: saved_model/assets


In [ ]:
save_model

In [ ]:
retval = cv.dnn.readNetFromTensorflow('saved_model')

In [7]:
import numpy as np
import tensorflow as tf
import cv2 as cv

# Read the graph.
with tf.io.gfile.FastGFile('saved_model/saved_model.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Session() as sess:
    # Restore session
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

    # Read and preprocess an image.
    img = cv.imread('neg/332.jpg')
    rows = img.shape[0]
    cols = img.shape[1]
    inp = cv.resize(img, (300, 300))
    inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

    # Run the model
    out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                    sess.graph.get_tensor_by_name('detection_scores:0'),
                    sess.graph.get_tensor_by_name('detection_boxes:0'),
                    sess.graph.get_tensor_by_name('detection_classes:0')],
                   feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

    # Visualize detected bounding boxes.
    num_detections = int(out[0][0])
    for i in range(num_detections):
        classId = int(out[3][0][i])
        score = float(out[1][0][i])
        bbox = [float(v) for v in out[2][0][i]]
        if score > 0.3:
            x = bbox[1] * cols
            y = bbox[0] * rows
            right = bbox[3] * cols
            bottom = bbox[2] * rows
            cv.rectangle(img, (int(x), int(y)), (int(right), int(bottom)), (125, 255, 51), thickness=2)

cv.imshow('TensorFlow MobileNet-SSD', img)
cv.waitKey()

AttributeError: module 'tensorflow._api.v2.io.gfile' has no attribute 'FastGFile'

In [13]:
import tensorflow as tf
import numpy as np

graph_filename = "saved_model/saved_model.pb"
with tf.compat.v2.io.gfile.GFile(graph_filename, "rb")as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    tf.train.write_graph(graph_def, './', 'frozen_graph.pbtxt', True)

DecodeError: Error parsing message

In [17]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
loaded = tf.saved_model.load('ModelWeights.h5')
infer = loaded.signatures['serving_default']
f = tf.function(infer).get_concrete_function(flatten_input=tf.TensorSpec(shape=[None, 28, 28, 1], dtype=tf.float32))
f2 = convert_variables_to_constants_v2(f)
graph_def = f2.graph.as_graph_def()
# Export frozen graph
with tf.io.gfile.GFile('frozen_graph.pb', 'wb') as f:
   f.write(graph_def.SerializeToString())

2.4.0-rc0


OSError: SavedModel file does not exist at: ModelWeights.h5/{saved_model.pbtxt|saved_model.pb}

In [19]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [20]:
freeze_session(session='ModelWeights.h5', output_names='Saved_model2')

AttributeError: 'str' object has no attribute 'graph'

In [27]:
import numpy as np
import tensorflow as tf
import cv2 as cv

# Read the graph.
with tf.gfile.FastGFile('saved_model/saved_model.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

with tf.Session() as sess:
    # Restore session
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

    # Read and preprocess an image.
    img = cv.imread('img_85.jpg')
    rows = img.shape[0]
    cols = img.shape[1]
    inp = cv.resize(img, (300, 300))
    inp = inp[:, :, [2, 1, 0]]  # BGR2RGB

    # Run the model
    out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                    sess.graph.get_tensor_by_name('detection_scores:0'),
                    sess.graph.get_tensor_by_name('detection_boxes:0'),
                    sess.graph.get_tensor_by_name('detection_classes:0')],
                   feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})

    # Visualize detected bounding boxes.
    num_detections = int(out[0][0])
    for i in range(num_detections):
        classId = int(out[3][0][i])
        score = float(out[1][0][i])
        bbox = [float(v) for v in out[2][0][i]]
        if score > 0.3:
            x = bbox[1] * cols
            y = bbox[0] * rows
            right = bbox[3] * cols
            bottom = bbox[2] * rows
            cv.rectangle(img, (int(x), int(y)), (int(right), int(bottom)), (125, 255, 51), thickness=2)

cv.imshow('TensorFlow MobileNet-SSD', img)
cv.waitKey()

AttributeError: module 'tensorflow' has no attribute 'gfile'